In [1]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Loading the predictions

In [2]:
df = pd.read_csv(r"C:\Users\neupa\Downloads\Final_test_Predictions.csv")

In [3]:
df

,title,text,label,title_tokens,text_tokens,text_summary,input,predicted_labels,predicted_probability_0,predicted_probability_1
0,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,1,13,387,NaN,House Intelligence Committee Chairman Devin Nu...,1,0.000203,0.999797
1,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,1,13,477,NaN,Pope Francis used his annual Christmas Day mes...,1,0.000234,0.999766
2,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,1,18,367,NaN,The number of cases of cops brutalizing and ki...,1,0.000186,0.999814
3,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,1,19,481,NaN,Donald Trump spent a good portion of his day a...,1,0.000186,0.999814
4,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,1,19,454,NaN,In the wake of yet another court decision that...,1,0.000369,0.999631
...,...,...,...,...,...,...,...,...,...,...
44893,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,0,19,889,North Korea and the US clashed at a U.N. forum...,North Korea and the US clashed at a U.N. forum...,0,0.999795,0.000205
44894,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,0,19,889,North Korea and the United States clashed at a...,North Korea and the United States clashed at a...,0,0.999764,0.000236
44895,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,0,20,810,North Korea and the US clashed at the United N...,North Korea and the US clashed at the United N...,0,0.999518,0.000482
44896,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,0,17,554,Two North Korean shipments to a Syrian governm...,Two North Korean shipments to a Syrian governm...,0,0.999425,0.000575
